In [1]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
from datetime import datetime
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import utils as util

In [2]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [4]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, StreetSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2022-01-01T00:00:00.000Z',
        'Maximum': '2023-02-27T15:43:16.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [5]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [6]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [7]:
channelInfoData = channelResponse.json()
data = response.json()

In [8]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

# Put it into function 
We wrap the data request into a nice reusable function and store it under the src folder and cann simply call it in the next step.

In [ ]:
# get all data from camera one
camera_one_data = util.return_data()

In [11]:
# get all datasets from camera two
camera_two_data_1 = util.return_data(site = "CameraTwo", scene = "SmallFieldSlide", all_coordinates = True)
camera_two_data_2 = util.return_data(site = "CameraTwo", scene = "StreetSlide", all_coordinates = True)
camera_two_data_3 = util.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)

In [24]:
# merge all four datasets and sort by 
df_all = pd.concat([camera_one_data, camera_two_data_1, camera_two_data_2, camera_two_data_3], 
                   keys=["CameraOne_HospitalView", "CameraTwo_SmallFieldSlide", "CameraTwo_StreetSlide", "CameraTwo_SchoolView"])

In [25]:
df_all

Timestamp  \
CameraOne_HospitalView 0    2023-01-18T15:47:13.68219+00:00   
                       1   2023-01-18T16:00:00.026914+00:00   
                       2   2023-01-19T09:00:00.464237+00:00   
                       3   2023-01-19T16:00:00.012271+00:00   
                       4   2023-01-20T09:00:00.005397+00:00   
...                                                     ...   
CameraTwo_SchoolView   66  2023-02-25T06:00:00.011158+00:00   
                       67  2023-02-25T14:00:00.001246+00:00   
                       68  2023-02-26T06:00:00.003552+00:00   
                       69  2023-02-26T14:00:00.009312+00:00   
                       70  2023-02-27T06:00:00.010685+00:00   

                                                                   ImageUrls  \
CameraOne_HospitalView 0   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       1   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       2   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       3   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       4   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
...                                                                      ...   
CameraTwo_SchoolView   66  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       67  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       68  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       69  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       70  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   

                          ACTUAL_TEMPERATURE HUMIDITY ILLUMINATION  \
CameraOne_HospitalView 0                17.1     67.0       2799.0   
                       1                17.1     67.0  1324.099976   
                       2           19.799999     52.0     100000.0   
                       3                17.5     71.0        122.0   
                       4                18.4     70.0       5659.0   
...                                      ...      ...          ...   
CameraTwo_SchoolView   66          16.299999     77.0       2054.0   
                       67          16.299999     77.0       2054.0   
                       68          16.299999     77.0       2054.0   
                       69          16.299999     77.0       2054.0   
                       70          16.299999     77.0       2054.0   

                          RAIN_COUNTER SUNSHINEDURATION WIND_SPEED  \
CameraOne_HospitalView 0         736.0            218.0       13.2   
                       1         736.0            218.0       10.6   
                       2    751.400024            487.0  39.700001   
                       3    751.400024            850.0        9.5   
                       4    753.099976           1122.0  16.299999   
...                                ...              ...        ...   
CameraTwo_SchoolView   66   814.200012           5558.0        7.0   
                       67   814.200012           5558.0        7.0   
                       68   814.200012           5558.0        7.0   
                       69   814.200012           5558.0        7.0   
                       70   814.200012           5558.0        7.0   

                          WIND_SPEED_CALC Rain.Day.Counter Rain.Diff  \
CameraOne_HospitalView 0         3.666667              0.0       0.0   
                       1         2.944444              0.0       0.0   
                       2        11.027778              0.0       0.0   
                       3         2.638889              0.0       0.0   
                       4         4.527778              0.0       0.0   
...                                   ...              ...       ...   
CameraTwo_SchoolView   66        1.944444              5.3       0.0   
                       67        1.944444              5.3       0.0   
  

In [ ]:
# save data
df_all.to_csv(index=False)

# To-Do
* allow for coordinate input in query function
* allow to query all different datasets for each site and each scence and combine